In [1]:
import time
import adodbapi
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
client_manuf =["Edgewell"]
client_brands = ["Hawaiian Tropic","Banana Boat","Beauty Care"]

decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
ManufOn=False

prodORitem = "SKU"
categories = ["Sun Care"]
# sectors = ["Sunscreen","Banded Pack","Tanning","After Sun"]
sectors = ["Sunscreen"]
segments = ["Sunscreen Cosmetics Aceite","Sunscreen Cosmetics Aerosol","Sunscreen Cosmetics Crema","Sunscreen Cosmetics Emulsion","Sunscreen Cosmetics Fluido","Sunscreen Cosmetics Gel","Sunscreen Cosmetics Liquido","Sunscreen Cosmetics Locion","Sunscreen Cosmetics Roll-On","Sunscreen Cosmetics Serum","Sunscreen Cosmetics Spray","Sunscreen Cosmetics Stick","Sunscreen Derma Aceite","Sunscreen Derma Crema","Sunscreen Derma Emulsion","Sunscreen Derma Fluido","Sunscreen Derma Gel","Sunscreen Derma Gel-Crema","Sunscreen Derma Liquido","Sunscreen Derma Locion","Sunscreen Derma Mousse","Sunscreen Derma Spray","Sunscreen Derma Stick","Sunscreen Sport Crema","Sunscreen Sport Liquido","Sunscreen Sport Locion","Sunscreen Sport Roll-On","Sunscreen Sport Spray","Sunscreen Sport Stick"]

# segments = ["Sunscreen Aceite","Sunscreen Aerosol","Sunscreen Crema","Sunscreen Emulsion","Sunscreen Fluido","Sunscreen Gel","Sunscreen Gel-Crema","Sunscreen Liquido","Sunscreen Locion","Sunscreen Mousse","Sunscreen Roll-On","Sunscreen Serum","Sunscreen Spray","Sunscreen Stick"]

# segments = ["Sunscreen Adults","Sunscreen Baby","Sunscreen Kids"]

subsegments= []
subcategories= []
 
national = True
customareas= "AUTOS SCANNING"
areas = ["NATIONAL","CHANNEL",f'{customareas}']#["RETAILER"]
 
regions_RET  = []
channels_RET = []
market_RET = []

regions_CHAN = ["Canal Moderno", "Convenience + Farmacias Cad + Hard Discounters",	"Canal Tradicional"]
channels_CHAN = []
market_CHAN = []
 
regions_CUST = ["Autos Scanning"]
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | April  2025"
years = {2023,2024,2025}


ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
end_date = "2025-05-01"

past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
National=["NATIONAL"]if national else []
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST
brands_only = True  # Get the Data of SKU Share by brands level only

entity_hierarchy = [
    ("Area",National),
    ("Region", regions),
    ("Channel", channels),
    ("Market", markets)
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
 
]
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

server = "powerbi://api.powerbi.com/v1.0/myorg/Edgewell"
dataset_name = "Edgewell Mexico Suncare Dataset"
conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=600;"
print(conn_str)

brackets = ['Base Price Bracket[Base Price Bracket]','Products[Pack Size Bracket]']

ISDcolumn='Total Size'# default should be 'Total Size'
path=os.path.join(os.getcwd(),"PPA NewEX")

# Format months for DAX
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"

Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Edgewell;Initial Catalog=Edgewell Mexico Suncare Dataset;Timeout=600;


In [3]:
start_time = time.time()

##### TopLine By Brand

In [4]:
def execute_dax_query(BrandorManuf,entity_name, area, hierby, filter_p12m, entity_type, bracket):
    outputdic = {}

    time_filter = f"""
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_type} | {entity_name}"
    filters = ", ".join(filter(None, [time_filter]))

    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()

    if table_name == 'Base Price Bracket':
        rowlst = "'Base Price Bracket','Base Price Bracket'[Base Price Bracket]"
        summarize_args = f"'Base Price Bracket'[Base Price Bracket], Products[{BrandorManuf}]"
    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"
        summarize_args = f"'{table_name}'[{column_name}], Products[{BrandorManuf}]"

    if BrandorManuf==f'{BrandOrTopB}':
        columns = ["Value Share", "Brand WoB %", "WoB %", "Value Sales IYA", "Relative Price"]
    else:
        columns = ["Value Share", "Company WoB %", "WoB %", "Value Sales IYA", "Relative Price"]

    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Main query
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{BrandorManuf}]))
                    ),
                    {summarize_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Totals query: uses VALUES to simulate total values per column
    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),  -- dummy base just to return a single row
                "Value Share", COALESCE([Value Share], 0),
                "Value Sales", COALESCE([Value Sales], 0),
                "Company WoB %", COALESCE([Brand WoB %], 0),  -- Assuming this is Brand WoB %
                "Relative Price", COALESCE([Relative Price], 0),
                "Value Sales IYA", COALESCE([Value Sales IYA], 0)
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute main query
            cursor.execute(main_dax_query)
            main_columns = [desc[0] for desc in cursor.description]
            main_data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute total query
            cursor.execute(total_dax_query)
            total_columns = [desc[0] for desc in cursor.description]
            total_data = cursor.fetchall()
            
        df = pd.DataFrame(main_data, columns=main_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        if df.empty:
                outputdic[key] = df
                return outputdic
        if not df.empty:
            grouped = []
            numeric_cols = df.select_dtypes(include='number').columns

            for group_name, group_df in df.groupby(df.columns[0]):
                grouped.append(group_df)

                subtotal_values = {
                    col: group_df[col].mean() if "price" in col.lower() else group_df[col].sum()
                    for col in numeric_cols
                }
                subtotal_row = {df.columns[0]: f"{group_name} Total"}
                subtotal_row.update(subtotal_values)
                grouped.append(pd.DataFrame([subtotal_row]))

            df_with_totals = pd.concat(grouped, ignore_index=True)

            # Process grand total row
            if total_data:
                dt = pd.DataFrame([total_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in total_columns])
                grand_total_row = {df.columns[0]: "Grand Total"}
                for col in numeric_cols:
                    if col in dt.columns:
                        grand_total_row[col] = dt[col].values[0]
                df_with_totals = pd.concat([
                    df_with_totals,
                    pd.DataFrame([grand_total_row], columns=df.columns)
                ], ignore_index=True)

        outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic

def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels, time_filter, brackets):
    os.makedirs(path, exist_ok=True)  # Ensure output directory exists

    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            dfs_results = {}  # Reset dfs_results for each bracket!
            futures = {}
            ordered_keys=[]
            for hierby, hier_values in hierarchy_levels:
                    if isinstance(hier_values, list):
                        for value in hier_values:
                            for area, entity_list in entity_hierarchy:
                                for entity in entity_list:
                                    # print(hierby,value,entity)                                    
                                    key = f"{value} | {entity}"
                                    ordered_keys.append(key)
                                    future = executor.submit(execute_dax_query,BrandorManuf, entity, area, hierby, time_filter, value, bracket)
                                    futures[future] = key
            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            # Insert results in original order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            if BrandorManuf==f'{BrandOrTopB}':
                filename = f"share_topline_{column_name}".replace(" ", "_").replace("\xa0", "_").lower()
            else:
                filename = f"share_topline_{column_name}_manuf".replace(" ", "_").replace("\xa0", "_").lower()

            output_file = os.path.join(path, f"{filename}.pkl")

            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"Saved data for {bracket} to {output_file}.")

# Example function call
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)
if ManufOn:
    process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)



Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket.pkl.
Saved data for Products[Pack Size Bracket] to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_pack_size_bracket.pkl.


In [5]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        futures = {}
        for hierby, hier_values in hierarchy_levels:
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            filename = f"share_topline_{column_name}_manuf".replace(" ", "_").replace("\xa0", "_").lower()
            output_file = os.path.join(path, f"{filename}.pkl")
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)


##### CatScope By Brand

In [6]:
def execute_dax_query(BrandorMaunf,entity_name, area, hierby, filter_p12m, bracket):
    outputdic = {}
    
    time_filter = f"""
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_name}"
    filters = ", ".join(filter(None, [time_filter]))
    
    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()
    
    if table_name == 'Base Price Bracket':
        # Correct format for SUMMARIZE arguments
        rowlst = "'Base Price Bracket','Base Price Bracket'[Base Price Bracket]"
        summarize_args = f"'Base Price Bracket'[Base Price Bracket],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizetotal_args = f"'Base Price Bracket'[Base Price Bracket],Products[{hierby}]"

    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"
        summarize_args = f"'{table_name}'[{column_name}],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizetotal_args = f"'{table_name}'[{column_name}],Products[{hierby}]"
        
    if BrandorMaunf==f'{BrandOrTopB}':
        columns = ["Value Share", "Brand WoB %", "WoB %"]
    else:
        columns = ["Value Share", 'Company WoB %', "WoB %"]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Construct the DAX query with correct SUMMARIZE syntax
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{BrandorMaunf}]))
                    ),
                    {summarize_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    grandtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),  -- dummy base just to return a single row
                {column_exprs}

            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    maintotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}]))
                    ),
                    {summarizetotal_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """

    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    DISTINCT(SELECTCOLUMNS({rowlst})),
                    '{table_name}'[{column_name}]
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    try:
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute main query
                cursor.execute(main_dax_query)
                main_columns = [desc[0] for desc in cursor.description]
                main_data = cursor.fetchall()

            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute total query
                cursor.execute(grandtotal_dax_query)
                grandtotal_columns = [desc[0] for desc in cursor.description]
                grandtotal_data = cursor.fetchall()
                
            # Execute main query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(maintotal_dax_query)
                maintotal_columns = [desc[0] for desc in cursor.description]
                maintotal_data = cursor.fetchall()

            # Execute total query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(total_dax_query)
                total_columns = [desc[0] for desc in cursor.description]
                total_data = cursor.fetchall()
                
            maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
            maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]

            total_df = pd.DataFrame(total_data, columns=total_columns)
            total_df.columns = total_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            total_df = total_df.loc[~(total_df.select_dtypes(include='number') == 0).all(axis=1)]
            if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 1] = maintotal_df.iloc[:, 1].astype(str) + " total"

            # ✅ Prepend "total " to column 0 in total_df
            if total_df.shape[1] > 0:
                total_df.iloc[:, 0] = total_df.iloc[:, 0].astype(str) +" total" 

            if maintotal_df.empty:
                outputdic[key] = maintotal_df
                return outputdic
            if not maintotal_df.empty:
                df_with_totals = pd.concat([maintotal_df, total_df], ignore_index=True)
        
            df = pd.DataFrame(main_data, columns=main_columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

            if df.empty:
                    outputdic[key] = df
                    return outputdic
            if not df.empty:
                grouped = []
                numeric_cols = df.select_dtypes(include='number').columns

                # Process grand total row
                if grandtotal_data:
                    dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
                    grand_total_row = {df.columns[0]: "Grand Total"}
                    for col in numeric_cols:
                        if col in dt.columns:
                            grand_total_row[col] = dt[col].values[0]
                    df_with_totals = pd.concat([
                        df,df_with_totals,
                        pd.DataFrame([grand_total_row], columns=df.columns)
                    ], ignore_index=True)

            outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic

def process_dax_queries(BrandorMaunf,entity_hierarchy, hierarchy_levels, time_filter, brackets):
    os.makedirs(path, exist_ok=True)  # Ensure output directory exists

    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            dfs_results = {}  # Reset dfs_results for each bracket!
            futures = {}
            ordered_keys = []

            for hierby, hier_values in hierarchy_levels:
                if hierby=="Category":
                    continue
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key = f"{entity}"  # Match key used in `execute_dax_query`
                        ordered_keys.append(key)
                        future = executor.submit(execute_dax_query,BrandorMaunf, entity, area, hierby, time_filter, bracket)
                        futures[future] = key

                temp_results = {}
                for future in as_completed(futures):
                    result = future.result()
                    temp_results.update(result)

                # Insert results in original order
                for key in ordered_keys:
                    if key in temp_results:
                        dfs_results[key] = temp_results[key]
                # Save results only for the current bracket
                column_name = bracket.split("[")[-1].rstrip("]")
                if BrandorMaunf==f'{BrandOrTopB}':
                    filename = f"share_topline_{column_name}_catscope_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
                else:
                    filename = f"share_topline_{column_name}_catscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()

                output_file = os.path.join(path, f"{filename}.pkl")

                with open(output_file, "wb") as f:
                    pd.to_pickle(dfs_results, f)

                print(f"Saved data for {bracket} to {output_file}.")

# Example function call
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)
if ManufOn:
    process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels, time_filter=True, brackets=brackets)



Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_sector.pkl.
Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_segment.pkl.
Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_subsegment.pkl.
Saved data for Base Price Bracket[Base Price Bracket] to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_catscope_subcategory.pkl.
Saved data for Products[Pack Size 

### if ManufOn==False: Create empty  pickle files 


In [7]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        futures = {}
        for hierby, hier_values in hierarchy_levels:
            if hierby=="Category":
                continue
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            filename = f"share_topline_{column_name}_catscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
            output_file = os.path.join(path, f"{filename}.pkl")
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)


### Parent Scope By Brand

In [8]:
def execute_dax_query(BrandorMaunf,entity_name, area, hierby,direct_parent, filter_p12m, bracket):
    outputdic = {}
    
    time_filter = f"""
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P12M"
        )
    """ if filter_p12m else ""

    key = f"{entity_name}"
    filters = ", ".join(filter(None, [time_filter]))
    
    table_name, column_name = bracket.split("[")
    table_name = table_name.strip()
    column_name = column_name.rstrip("]").strip()
    
    if table_name == 'Base Price Bracket':
        # Correct format for SUMMARIZE arguments
        rowlst = "'Base Price Bracket','Base Price Bracket'[Base Price Bracket]"
        summarize_args = f"'Base Price Bracket'[Base Price Bracket], Products[{direct_parent[hierby]}],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizeparenttot_args = f"'Base Price Bracket'[Base Price Bracket], Products[{direct_parent[hierby]}]"
        summarizetotal_args = f"'Base Price Bracket'[Base Price Bracket], Products[{direct_parent[hierby]}],Products[{hierby}]"
        

    else:
        rowlst = f"{table_name},{table_name}[{column_name}]"
        summarize_args = f"'{table_name}'[{column_name}], Products[{direct_parent[hierby]}],Products[{hierby}], Products[{BrandorMaunf}]"
        summarizeparenttot_args = f"'{table_name}'[{column_name}], Products[{direct_parent[hierby]}]"
        summarizetotal_args = f"'{table_name}'[{column_name}], Products[{direct_parent[hierby]}],Products[{hierby}]"
        

    if BrandorMaunf==f'{BrandOrTopB}':
        columns = ["Value Share", "Brand WoB %", "WoB %","Value Sales"]
    else:            
        columns = ["Value Share",'Company WoB %', "WoB %","Value Sales"]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    # Construct the DAX query with correct SUMMARIZE syntax
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{direct_parent[hierby]}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{BrandorMaunf}]))
                    ),
                    {summarize_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )   
    """
    grandtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),  -- dummy base just to return a single row
                {column_exprs}

            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{direct_parent[hierby]}]))
                    ),
                    {summarizeparenttot_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """
    childtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    CROSSJOIN(
                        DISTINCT(SELECTCOLUMNS({rowlst})),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{direct_parent[hierby]}])),
                        DISTINCT(SELECTCOLUMNS(Products, Products[{hierby}]))

                        
                    ),
                    {summarizetotal_args}
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """

    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    DISTINCT(SELECTCOLUMNS({rowlst})),
                    '{table_name}'[{column_name}]
                ), 
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    try:
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute main query
                cursor.execute(main_dax_query)
                main_columns = [desc[0] for desc in cursor.description]
                main_data = cursor.fetchall()

            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                # Execute total query
                cursor.execute(grandtotal_dax_query)
                grandtotal_columns = [desc[0] for desc in cursor.description]
                grandtotal_data = cursor.fetchall()
                
            # Execute main query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_columns = [desc[0] for desc in cursor.description]
                maintotal_data = cursor.fetchall()
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(childtotal_dax_query)
                childtotal_columns = [desc[0] for desc in cursor.description]
                childtotal_data = cursor.fetchall()

            # Execute total query
            with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
                cursor.execute(total_dax_query)
                total_columns = [desc[0] for desc in cursor.description]
                total_data = cursor.fetchall()
                
            maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
            maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
            
            childtotal_df = pd.DataFrame(childtotal_data, columns=childtotal_columns)
            childtotal_df.columns = childtotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            childtotal_df = childtotal_df.loc[~(childtotal_df.select_dtypes(include='number') == 0).all(axis=1)]


            total_df = pd.DataFrame(total_data, columns=total_columns)
            total_df.columns = total_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            total_df = total_df.loc[~(total_df.select_dtypes(include='number') == 0).all(axis=1)]
            
            if childtotal_df.shape[1] > 0:
                childtotal_df.iloc[:, 2] = childtotal_df.iloc[:, 2].astype(str) +" total" 
                
            if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 1] = maintotal_df.iloc[:, 1].astype(str) + " total"

            # ✅ Prepend "total " to column 0 in total_df
            if total_df.shape[1] > 0:
                total_df.iloc[:, 0] = total_df.iloc[:, 0].astype(str) +" total" 
                


            if maintotal_df.empty:
                outputdic[key] = maintotal_df
                return outputdic
            if not maintotal_df.empty:
                df_with_totals = pd.concat([maintotal_df,childtotal_df, total_df], ignore_index=True)
        
            df = pd.DataFrame(main_data, columns=main_columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

            if df.empty:
                    outputdic[key] = df
                    return outputdic
            if not df.empty:
                grouped = []
                numeric_cols = df.select_dtypes(include='number').columns

                # Process grand total row
                if grandtotal_data:
                    dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
                    grand_total_row = {df.columns[0]: "Grand Total"}
                    for col in numeric_cols:
                        if col in dt.columns:
                            grand_total_row[col] = dt[col].values[0]
                    df_with_totals = pd.concat([
                        df,df_with_totals,
                        pd.DataFrame([grand_total_row], columns=df.columns)
                    ], ignore_index=True)

            outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic


def process_dax_queries(BrandorMaunf,entity_hierarchy, hierarchy_levels, direct_parent, time_filter, brackets):
    os.makedirs(path, exist_ok=True)  # Ensure output directory exists

    with ThreadPoolExecutor(max_workers=8) as executor:
        for bracket in brackets:
            for hierby, hier_values in hierarchy_levels:
                if hierby == "Category":
                    continue

                print(f"Processing bracket: {bracket} | hierby: {hierby}")
                dfs_results = {}  # Reset for each combination of bracket + hierby
                futures = {}

                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        future = executor.submit(execute_dax_query,BrandorMaunf, entity, area, hierby, direct_parent, time_filter, bracket)
                        futures[future] = (entity, area)

                for future in as_completed(futures):
                    result = future.result()
                    dfs_results.update(result)

                column_name = bracket.split("[")[-1].rstrip("]")
                if BrandorMaunf==f"{ManufOrTopC}":
                    filename = f"share_topline_{column_name}_parentscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
                else:    
                    filename = f"share_topline_{column_name}_parentscope_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
                output_file = os.path.join(path, f"{filename}.pkl")

                with open(output_file, "wb") as f:
                    pd.to_pickle(dfs_results, f)

                print(f"✅ Saved data for bracket: {bracket}, hierby: {hierby} to {output_file}")


# Example function call
process_dax_queries(f"{BrandOrTopB}",entity_hierarchy, hierarchy_levels,direct_parent, time_filter=True, brackets=brackets)
if ManufOn:
    process_dax_queries(f"{ManufOrTopC}",entity_hierarchy, hierarchy_levels,direct_parent, time_filter=True, brackets=brackets)



Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: Sector
✅ Saved data for bracket: Base Price Bracket[Base Price Bracket], hierby: Sector to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_parentscope_sector.pkl
Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: Segment
✅ Saved data for bracket: Base Price Bracket[Base Price Bracket], hierby: Segment to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline_base_price_bracket_parentscope_segment.pkl
Processing bracket: Base Price Bracket[Base Price Bracket] | hierby: SubSegment
✅ Saved data for bracket: Base Price Bracket[Base Price Bracket], hierby: SubSegment to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\share_topline

### if ManufOn==False: Create empty  pickle files 

In [9]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        futures = {}
        for hierby, hier_values in hierarchy_levels:
            if hierby=="Category" or direct_parent[hierby]=="Category":
                continue
            # Save results only for the current bracket
            column_name = bracket.split("[")[-1].rstrip("]")
            filename = f"share_topline_{column_name}_parentscope_manuf_{hierby}".replace(" ", "_").replace("\xa0", "_").lower()
            output_file = os.path.join(path, f"{filename}.pkl")
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)


## Inter-size Discount

In [10]:
def execute_dax_query(entity_name, area, filter_p3m, brand=None):
    outputdic = {}
    
    time_filter = """
        FILTER(
            VALUES('Time Logic'[Time Period]), 
            'Time Logic'[Time Period] = "P3M"
        )
    """ if filter_p3m else ""

    brand_filter = f'Products[{BrandOrTopB}]="{brand}"' if brand else ""
    key = f"{categories[0]} | {brand} | {entity_name}" if brand else ""
    filters = ", ".join(filter(None, [brand_filter, time_filter]))

    # Define columns dynamically
    columns = [
        "Base Price/Unit","Value Sales", "Unit Sales","Value Sales IYA","VSOD"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    row_list=[]
    if ISDcolumn != 'Total Size':
        her_list = ["Products[Variant]", "Products[Total Size]", f"Products[{ISDcolumn}]"]
    else:
        her_list = ["Products[Variant]", "Products[Total Size]"]
    if her_list:
        summarize_expr = f"SUMMARIZE(Products, {', '.join(her_list)})"
        
    main_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                {summarize_expr},
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",            
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    total_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            {filters},
            Products[Category] = "{categories[0]}",
            TREATAS({{"{entity_name}"}}, Market[{area}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute main query
            cursor.execute(main_dax_query)
            main_columns = [desc[0] for desc in cursor.description]
            main_data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            # Execute total query
            cursor.execute(total_dax_query)
            total_columns = [desc[0] for desc in cursor.description]
            total_data = cursor.fetchall()
            
        df = pd.DataFrame(main_data, columns=main_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        if df.empty:
                outputdic[key] = df
                return outputdic
        if not df.empty:
            grouped = []
            numeric_cols = df.select_dtypes(include='number').columns

            for group_name, group_df in df.groupby(df.columns[0]):
                grouped.append(group_df)

                subtotal_values = {
                    col: group_df[col].mean() if "price" in col.lower() else group_df[col].sum()
                    for col in numeric_cols
                }
                subtotal_row = {df.columns[0]: f"{group_name} Total"}
                subtotal_row.update(subtotal_values)
                grouped.append(pd.DataFrame([subtotal_row]))

            df_with_totals = pd.concat(grouped, ignore_index=True)

            # Process grand total row
            if total_data:
                dt = pd.DataFrame([total_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in total_columns])
                grand_total_row = {df.columns[0]: "Grand Total"}
                for col in numeric_cols:
                    if col in dt.columns:
                        grand_total_row[col] = dt[col].values[0]
                df_with_totals = pd.concat([
                    df_with_totals,
                    pd.DataFrame([grand_total_row], columns=df.columns)
                ], ignore_index=True)

        outputdic[key] = df_with_totals if not df.empty else df

    except Exception as e:
        print(f"Error processing {entity_name}: {str(e)}")
        print(f"DAX Query:\n{main_dax_query}")

    return outputdic



def process_dax_queries(entity_hierarchy, time_filter, client_brands=None):
    dfs_results = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        for area, entity_list in entity_hierarchy:
            for entity in entity_list:                    
                if client_brands:
                    for brand in client_brands:
                        futures.append(executor.submit(execute_dax_query, entity, area, time_filter, brand=brand))

        for future in as_completed(futures):
            try:
                result = future.result()
                dfs_results.update(result)
            except Exception as e:
                print(f"Error processing query: {e}")

        # Construct the output file name correctly
        
        filename = "inter_size.pkl" 

        output_file = f"{path}\\{filename}"

        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)

        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy, time_filter=True,client_brands=client_brands)  # Brand-level

All DataFrames saved to c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\PPA slide duplicate\PPA NewEX\inter_size.pkl.


## if ManufOn==False: Create empty  pickle files 

In [11]:
if ManufOn==False:
    for bracket in brackets:
        dfs_results = {}  # Reset dfs_results for each bracket!
        column_name = bracket.split("[")[-1].rstrip("]")
        filename = f"share_topline_{column_name}_manuf".replace(" ", "_").replace("\xa0", "_").lower()
        output_file = os.path.join(path, f"{filename}.pkl")
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)


In [12]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Thu Jul  3 13:11:48 2025
Script ended at: Thu Jul  3 13:17:20 2025
Elapsed time: 331.25 seconds


In [13]:
loaded_data = {}
datasets_path =os.getcwd()+"\\PPA NewEX\\"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)